In [12]:
library(tidyverse)
library(broom)
library(repr)
library(digest)
library(infer)

## **Introduction**

Students who participate in extracurricular activities, such as regular artistic or physical activities, can gain additional transferable skills. Yet, there are concerns that this involvement may come at the cost of dedicating less time to academic studies, leading to negative performance in school. However, an analytical research paper by Ribeiro (2023) reveals that the majority of past clinical trials found a positive correlation between extracurricular activities and academic success. Extracurricular activities had benefits for mental health and suicide prevention among college students, contributing to better grades in school. Similarly, another study found that it enhances leadership, time management, and social skills, ultimately contributing to improved academic performance (Christison, 2013). A study by Wang (2023) found that students who participated in extracurricular activities received similar grades as students who did not, suggesting that participation in extracurricular activities does not compromise academic performance. Collectively, these studies consistently show a positive correlation between extracurricular activities and academic success. Our statistical analysis aims to confirm the validity of this trend.

## **Preliminary Results**

In [13]:
url <- "https://raw.githubusercontent.com/PeterYuLi1204/STAT-201-Project/main/StudentsPerformance_with_headers.csv" 
student_data <- read_csv(url) 
head(student_data)

New names:
• `Reading frequency` -> `Reading frequency...19`
• `Reading frequency` -> `Reading frequency...20`
Rows: 145 Columns: 33
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): STUDENT ID
dbl (32): Student Age, Sex, Graduated high-school type, Scholarship type, Ad...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


STUDENT ID,Student Age,Sex,Graduated high-school type,Scholarship type,Additional work,Regular artistic or sports activity,Do you have a partner,Total salary if available,Transportation to the university,⋯,Preparation to midterm exams 1,Preparation to midterm exams 2,Taking notes in classes,Listening in classes,Discussion improves my interest and success in the course,Flip-classroom,Cumulative grade point average in the last semester (/4.00),Expected Cumulative grade point average in the graduation (/4.00),COURSE ID,GRADE
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
STUDENT1,2,2,3,3,1,2,2,1,1,⋯,1,1,3,2,1,2,1,1,1,1
STUDENT2,2,2,3,3,1,2,2,1,1,⋯,1,1,3,2,3,2,2,3,1,1
STUDENT3,2,2,2,3,2,2,2,2,4,⋯,1,1,2,2,1,1,2,2,1,1
STUDENT4,1,1,1,3,1,2,1,2,1,⋯,1,2,3,2,2,1,3,2,1,1
STUDENT5,2,2,1,3,2,2,1,3,1,⋯,2,1,2,2,2,1,2,2,1,1
STUDENT6,2,2,2,3,2,2,2,2,1,⋯,1,1,1,2,1,2,4,4,1,2


## **Methods and Plan**

## **References**

Christison, C. (2013). The benefits of participating in extracurricular activities. BU Journal of Graduate Studies in Education, 5(2), 17.

Ribeiro, N., Malafaia, C., Neves, T., & Menezes, I. (2023). The impact of extracurricular activities on university students' academic success and employability. European Journal of Higher Education, ahead-of-print(ahead-of-print), 1-21. https://doi.org/10.1080/21568235.2023.2202874

Wang, D., Xiong, R., Zhang, J., Han, X., Jin, L., Liu, W., Qu, Y., Chen, Q., Chen, S., Chen, X., Li, Y., He, M., Zeng, Y., & Liu, Y. (2023). Effect of extracurricular after-school physical activities on academic performance of schoolchildren: A cluster randomized clinical trial. JAMA Pediatrics, https://doi.org/10.1001/jamapediatrics.2023.3615
